# 준비

In [1]:
import pandas as pd
import numpy as np
import os
import re

import nltk
from numpy.random import randint, seed
from sklearn.feature_extraction.text import CountVectorizer

import konlpy
from konlpy.tag import Twitter

from collections import Counter
from wordcloud import WordCloud

from PIL import Image  
from IPython.display import set_matplotlib_formats
from nltk.corpus import stopwords

In [2]:
os.getcwd()

'C:\\workspace\\Python\\4-2) 핀테크 교육\\Final Project\\Project1'

In [3]:
os.chdir(r".\data")
os.getcwd()

'C:\\workspace\\Python\\4-2) 핀테크 교육\\Final Project\\Project1\\data'

# Load data

In [4]:
my_text=pd.read_csv("SH_card.csv", encoding='CP949')
my_text.shape

(293, 2)

In [5]:
my_text['text'].values[:10]

array(['신한 FAN 이용금액별 포인트 적립 이벤트', '신한카드 행복드림(Dream) 페스티벌!',
       '워너원 한정판 체크카드 사용하고 경품받자!', 'L7호텔 Yellow Summer 이벤트',
       '파크 하얏트 서울 Summer Staycation 이벤트', '「신한카드 골프 챌린지」참가 신청 안내',
       '해외여행 최대 5% + 최대 10만원할인', 'PEUGEOT 2008 SUV Zero 이벤트',
       '시트로엥 C4 CACTUS Zero 이벤트', '신한카드 YOOX 10~15% 추가할인'], dtype=object)

# 전처리

In [6]:
len(my_text['text'].values) - len(set(my_text['text'].values)) # 중복 데이터

49

In [7]:
no_meaning = "이벤트|신한카드|신한"
my_text_clean = []

for a_line in set(my_text['text'].values):
    a_line = re.sub('[^ㄱ-ㅣ가-힣]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)           # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)               # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [8]:
my_text_clean[:10]

[' 추가할인',
 '부산 송도해상케이블카 할인 ',
 ' 교통카드 캐시백 ',
 '롯데월드 키즈파크에서 마이포인트 사용하세요',
 '환경부 기념 전기차 충전 하이패스 전액지원 ',
 '올댓컬쳐 뮤지컬 최대 할인',
 ' 개편맞이 퀴즈 경품 ',
 '도시가스 자동이체신청 최대 원 캐시백 ',
 '신세계백화점에서 결제 시 신세계상품권 증정 월 ',
 '해외쇼핑 전상품 추가할인']

In [9]:
hannanum = konlpy.tag.Hannanum()

# 명사 추출
my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)
    
#my_words[:10]

In [10]:
# 단음절 제거
my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word
        
#my_words_2

# 2-gram

In [11]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [12]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [13]:
res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:20]

[('아고다 호텔', 8),
 ('창립 주년', 7),
 ('최대 캐시백', 6),
 ('하얏트 서울', 4),
 ('신세계상품권 증정', 3),
 ('아트 컬렉션', 3),
 ('연회비 캐시백', 3),
 ('올댓쇼핑 첫구', 3),
 ('주년 기념', 3),
 ('첫구 감사', 3),
 ('최대 포인트', 3),
 ('캐시백 창립', 3),
 ('캐시백 해외직구', 3),
 ('파크 하얏트', 3),
 ('포인트 적립', 3),
 ('플라자 호텔', 3),
 ('해외직구 최대', 3),
 ('감사 포인트적립', 2),
 ('개월 할부', 2),
 ('고객 감사', 2)]

In [14]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()           # A list of n-Grams.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])                         # (n-1)-Gram.
    next_word = words[-1]                                       # Word after the a_nm1_gram.
    if a_nm1_gram not in my_dict.keys():
        my_dict[a_nm1_gram] = [next_word]                       # a_nm1_gram is a new key. So, initialize the dictionary entry.
    else:
        my_dict[a_nm1_gram] += [next_word]                      # an_nm1_gram is already in the dictionary.

# 연관 단어 확인

In [15]:
def next_word (arr):
    for i in arr:
        print(i, "--->", my_dict[i][:5])

## 신한카드

In [16]:
next_word(['할인', '캐시백', '호텔', '포인트', '최대'])

할인 ---> ['개편맞이', '공연', '교통카드', '내차처럼', '더치페']
캐시백 ---> ['그랜드', '롯데월드', '롯데호텔', '받으', '분기']
호텔 ---> ['가든페스트', '라오스', '롯데호텔', '베트남', '부산']
포인트 ---> ['국내', '네이버페', '드림', '받으', '적립']
최대 ---> ['금액대별', '만원', '만원할인', '선착순', '설맞']


## 삼성카드

In [17]:
my_text=pd.read_csv("SS_card.csv")

In [18]:
no_meaning = "이벤트|삼성카드|삼성"
my_text_clean = []

for a_line in set(my_text['text'].values):
    a_line = re.sub('[^ㄱ-ㅣ가-힣]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)           # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)               # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [19]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)
    
my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [20]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [21]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [22]:
res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('만원 캐시백', 11),
 ('여행 혜택', 10),
 ('혜택 여행', 9),
 ('호텔 혜택', 8),
 ('가연 미팅파티', 6),
 ('창립 주년', 6),
 ('호텔 예약', 6),
 ('화재 다이렉트', 6),
 ('겨울 여행', 5),
 ('다이렉트 오토', 5)]

In [23]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()           # A list of n-Grams.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])                         # (n-1)-Gram.
    next_word = words[-1]                                       # Word after the a_nm1_gram.
    if a_nm1_gram not in my_dict.keys():
        my_dict[a_nm1_gram] = [next_word]                       # a_nm1_gram is a new key. So, initialize the dictionary entry.
    else:
        my_dict[a_nm1_gram] += [next_word]                      # an_nm1_gram is already in the dictionary.

In [24]:
def next_word (arr):
    for i in arr:
        print(i, "--->", my_dict[i][:5])

In [25]:
next_word(['캐시백', '여행', '보험'])

캐시백 ---> ['가연', '가족', '갤럭시', '경품', '교보라이프플래닛생명']
여행 ---> ['가뿐', '가을여행', '계절', '고민', '고속도로']
보험 ---> ['상식', '상품권', '연금저축보험', '캐시백']


## 국민카드

In [26]:
my_text=pd.read_csv("KB_card.csv")

In [27]:
no_meaning = "이벤트|국민카드|국민"
my_text_clean = []

for a_line in set(my_text['text'].values):
    a_line = re.sub('[^ㄱ-ㅣ가-힣]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)           # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)               # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [28]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)
    
my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [29]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [30]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [31]:
res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('최대 만원', 49),
 ('만원 캐시백', 46),
 ('최대 할인', 43),
 ('국제선 항공권', 20),
 ('항공권 최대', 20),
 ('개월 무이자할부', 15),
 ('라이프샵 하나', 12),
 ('호남 충청', 12),
 ('만원 할인', 11),
 ('온라인 신규발급', 11)]

In [32]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()           # A list of n-Grams.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])                         # (n-1)-Gram.
    next_word = words[-1]                                       # Word after the a_nm1_gram.
    if a_nm1_gram not in my_dict.keys():
        my_dict[a_nm1_gram] = [next_word]                       # a_nm1_gram is a new key. So, initialize the dictionary entry.
    else:
        my_dict[a_nm1_gram] += [next_word]                      # an_nm1_gram is already in the dictionary.

In [33]:
def next_word (arr):
    for i in arr:
        print(i, "--->", my_dict[i][:5])

In [34]:
next_word(['할인', '캐시백', '혜택'])

할인 ---> ['가정', '가정의달', '개월', '겨울', '고객대상']
캐시백 ---> ['가격보고', '가온부울경카드', '가족', '간편결제', '개인사업자']
혜택 ---> ['가을가을해', '가즈아', '개월', '게임', '경품']


## 우리카드

In [35]:
my_text=pd.read_csv("WR_card.csv")

In [36]:
no_meaning = "이벤트|우리카드|우리"
my_text_clean = []

for a_line in set(my_text['text'].values):
    a_line = re.sub('[^ㄱ-ㅣ가-힣]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)           # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)               # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [37]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)
    
my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [38]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [39]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [40]:
res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('최대 할인', 8),
 ('아고다 최대', 7),
 ('카드 정석', 5),
 ('최대 만원', 4),
 ('공사 임대주택', 3),
 ('만원 캐시백', 3),
 ('임대료 자동납부', 3),
 ('임대주택 임대료', 3),
 ('지방세 납부', 3),
 ('티켓 할인', 3)]

In [41]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()           # A list of n-Grams.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])                         # (n-1)-Gram.
    next_word = words[-1]                                       # Word after the a_nm1_gram.
    if a_nm1_gram not in my_dict.keys():
        my_dict[a_nm1_gram] = [next_word]                       # a_nm1_gram is a new key. So, initialize the dictionary entry.
    else:
        my_dict[a_nm1_gram] += [next_word]                      # an_nm1_gram is already in the dictionary.

In [42]:
def next_word (arr):
    for i in arr:
        print(i, "--->", my_dict[i][:5])

In [43]:
next_word(['할인', '납부', '캐시백'])

할인 ---> ['가정', '고객만', '공사', '구글플레', '대학']
납부 ---> ['실속', '청구할인', '캐시백']
캐시백 ---> ['경품', '도시가스요금', '사회보험료', '세븐일레븐', '아고다']


## 하나카드

In [44]:
my_text=pd.read_csv("HN_card.csv")

In [45]:
no_meaning = "이벤트|하나카드|하나"
my_text_clean = []

for a_line in set(my_text['text'].values):
    a_line = re.sub('[^ㄱ-ㅣ가-힣]+', ' ', a_line) # 한글, 숫자 이외는 모두 공백으로
    a_line = re.sub(no_meaning, '', a_line)           # 의미없는 문자 삭제
    a_line = re.sub('\s+', ' ', a_line)               # 잉여 스페이스를 공백 하나로
    my_text_clean += [a_line]

In [46]:
hannanum = konlpy.tag.Hannanum()

my_words = []
for a_line, n in zip(my_text_clean, range(len(my_text_clean))):
    #print(n, end=" ")
    if a_line == " ":
        #print("패스")
        continue
    my_words += hannanum.nouns(a_line)
    
my_words_2 = [""]
for a_word in my_words:
    if len(a_word) > 1:
        my_words_2[0] += " " + a_word

In [47]:
n = 2                                                            # Can be changed to a number equal or larger than 2.
n_min = n
n_max = n
n_gram_type = 'word'                                             # n-Gram with words.
vectorizer = CountVectorizer(ngram_range=(n_min,n_max), analyzer = n_gram_type)

In [48]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()            # Get the n-Grams as a list.
n_gram_cts = vectorizer.transform(my_words_2).toarray()             #  The output is an array of array.
n_gram_cts = list(n_gram_cts[0])                                 # Convert into a simple list.

In [49]:
res_dict = {}
for z, zz in zip(n_grams, n_gram_cts):
    res_dict[z] = zz
res_dict = sorted(res_dict.items(), key=(lambda x: x[1]), reverse=True)
res_dict[:10]

[('최대 할인', 11),
 ('쇼핑 제휴몰', 8),
 ('청구할인 혜택', 8),
 ('다이닝 서비스', 7),
 ('만원 써프라이즈', 7),
 ('문화 방법', 7),
 ('방법 월간', 7),
 ('월간 컬처', 7),
 ('체크 혜택', 6),
 ('국내 해외', 4)]

In [50]:
n_grams = vectorizer.fit(my_words_2).get_feature_names()           # A list of n-Grams.
my_dict = {}
for a_gram in n_grams:
    words = nltk.word_tokenize(a_gram)
    a_nm1_gram = ' '.join(words[0:n-1])                         # (n-1)-Gram.
    next_word = words[-1]                                       # Word after the a_nm1_gram.
    if a_nm1_gram not in my_dict.keys():
        my_dict[a_nm1_gram] = [next_word]                       # a_nm1_gram is a new key. So, initialize the dictionary entry.
    else:
        my_dict[a_nm1_gram] += [next_word]                      # an_nm1_gram is already in the dictionary.

In [51]:
def next_word (arr):
    for i in arr:
        print(i, "--->", my_dict[i][:5])

In [52]:
next_word(['할인', '혜택', '포인트'])

할인 ---> ['가족', '그랜드', '대표', '라로카', '롯데호텔']
혜택 ---> ['강력한', '경품', '라발레', '레진코믹스', '롯데워터파크']
포인트 ---> ['전환']
